# AMEX Export to Parquet with Apache Spark

<img src="https://img.freepik.com/free-vector/programming-development-isometric-composition-with-image-silicon-chip-with-human-character-code-screens-vector-illustration_1284-66482.jpg?t=st=1654010611~exp=1654011211~hmac=16433f991c4dae7041daed6cf9ae319b1c678dfa416b186ffd8b2b2a6902e0b9&w=1800" alt="clipart" width="500"/>
<sub><sup>Clipart Designed by Freepik</sup></sub>

### Hi! 👋

We will be using [Apache Spark](https://spark.apache.org) to export CSV files to a convenient Parquet format.

*The output of this code* will be a directory with Parquet files - they can be read by a) *pandas* and b) *Apache Spark*.

This notebook will show:

- importing CSV, exporting Parquet
- **an overview of column types** in AMEX dataset
- description of some Apache Spark characteristics on a true dataset

This code will:

1. Import raw CSV data
2. Preprocess data
3. Export to Parquet
4. Perform some trivial EDA

Resulting Parquet dataset will be published as **a Kaggle dataset**, so you can use it in your notebooks. 👍

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Installing Spark

Spark can be pip installed.

In [ ]:
!pip install pyspark==3.2.1

In [ ]:
import pyspark
import pyspark.sql.functions as F

pyspark.__version__

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[*]")\
    .config("spark.driver.memory", "14g")\
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 20)  # it isn't a cluster, so 20 is enough

spark

## 0. Save train_labels to Parquet

As a quick warmup, let's export `train_labels.csv` to Parquet! 🎣

I've choosen the smallest adequate column types - to save space.

In [ ]:
import pyspark.sql.types

# column types
schema = pyspark.sql.types.StructType([
    pyspark.sql.types.StructField('customer_ID', pyspark.sql.types.StringType()),
    pyspark.sql.types.StructField('target', pyspark.sql.types.ByteType())
])

df = (
    spark.read
    .schema(schema)
    .option("header", True)
    .option("mode", "FAILFAST")
    .csv("/kaggle/input/amex-default-prediction/train_labels.csv")
)

# interpreting 0 and 1 as false and true
df = df.withColumn(
    'target',
    F.col('target').cast(pyspark.sql.types.BooleanType())
    # F was earlier imported with "import ... as F"
)

df.show()

In [ ]:
# coalesce makes sure, that output is in 1 file
df.coalesce(1).write.parquet('amexparq/train_labels')

In [ ]:
ls -lh amexparq/train_labels

## 1. Import raw CSV data

Now we will get our hands dirty with training/test data!

The first step is to load raw CSV data.

### Column types 🎈

We have 5 types of columns:

||category|column names|description|example value|
|-|-|-|-|-|
|1.|Customer ID|`customer_ID`|string with hex digits|`'0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'`|
|2.|Datetime|`S_2`|*YYYY-MM-DD* datetime|`2017-03-09`|
|3.|Categorical columns, which are already numbers|`B_38`, `D_116`, ...|small integer represented as a float number|`2.0`|
|4.|Categorical columns, which have string labels|`D_63`, `D_64`|string labels from a fixed set|`'CR'`, `'CO'`|
|5.|Numerical columns|`R_1`, `D_58`, ...|float|`0.00922`|


Each of them must be properly handled. These columns can have *nulls*.

In [ ]:
# this cell is hidden
# because this list is quite long... 😕

all_columns = [
 'customer_ID',
 'S_2',
 'P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_42',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'B_6',
 'B_7',
 'B_8',
 'D_50',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'D_53',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'D_56',
 'B_13',
 'R_5',
 'D_58',
 'S_9',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_63',
 'D_64',
 'D_65',
 'B_16',
 'B_17',
 'B_18',
 'B_19',
 'D_66',
 'B_20',
 'D_68',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'D_73',
 'P_4',
 'D_74',
 'D_75',
 'D_76',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'R_9',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'D_82',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_29',
 'B_30',
 'S_18',
 'D_86',
 'D_87',
 'R_17',
 'R_18',
 'D_88',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 'R_22',
 'R_23',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'R_24',
 'R_25',
 'D_96',
 'S_22',
 'S_23',
 'S_24',
 'S_25',
 'S_26',
 'D_102',
 'D_103',
 'D_104',
 'D_105',
 'D_106',
 'D_107',
 'B_36',
 'B_37',
 'R_26',
 'R_27',
 'B_38',
 'D_108',
 'D_109',
 'D_110',
 'D_111',
 'B_39',
 'D_112',
 'B_40',
 'S_27',
 'D_113',
 'D_114',
 'D_115',
 'D_116',
 'D_117',
 'D_118',
 'D_119',
 'D_120',
 'D_121',
 'D_122',
 'D_123',
 'D_124',
 'D_125',
 'D_126',
 'D_127',
 'D_128',
 'D_129',
 'B_41',
 'B_42',
 'D_130',
 'D_131',
 'D_132',
 'D_133',
 'R_28',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_139',
 'D_140',
 'D_141',
 'D_142',
 'D_143',
 'D_144',
 'D_145',
]

In [ ]:
# schema definition (column types)

import pyspark.sql.types

# list "all_columns" is defined in a hidden cell above
print(f"len(all_columns)={len(all_columns)}")

categorical_columns = {
    'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68',
}
custom_label_columns = {'D_64', 'D_63'}  # these columns have string labels instead of integers

def get_type(col_name):
    """Get appriopriate type for given column."""
    
    # see https://spark.apache.org/docs/latest/sql-ref-datatypes.html

    if col_name == 'customer_ID':
        return pyspark.sql.types.StringType()
    elif col_name in categorical_columns:
        if col_name in custom_label_columns:
            # these columns have string labels instead of integers
            # we will later replace them with numbers
            return pyspark.sql.types.StringType()
        else:
            # these were written as floats, like "1.0"
            # we will later cast them to bytes
            return pyspark.sql.types.FloatType()
    elif col_name == 'S_2':
        # YYYY-MM-DD date type
        return pyspark.sql.types.DateType()
    else:
        return pyspark.sql.types.FloatType()

schema = pyspark.sql.types.StructType(
    [pyspark.sql.types.StructField(c, get_type(c)) for c in all_columns]
)


In [ ]:
import pyspark.sql.functions as F

# limit lines for debugging purposes
limit = None

def load_raw_data(path):
    # see https://spark.apache.org/docs/3.2.0/sql-data-sources-csv.html
    df = (
        spark.read
        .schema(schema)
        .option("header", True)
        .option("mode", "FAILFAST")
        .csv(path)
    )

    if limit is not None:
        df = df.limit(limit)
        
    return df

# notice, that in Spark dataframes are lazy-evaluated!
train_df = load_raw_data("/kaggle/input/amex-default-prediction/train_data.csv")
test_df = load_raw_data("/kaggle/input/amex-default-prediction/test_data.csv")

train_df

## 2. Preprocess data

There are a few problems with the raw data.

------

a) We had to parse some categorical columns as floats, because they were written with comma: `1.0`, `2.0`.

Byte would be better, because float has 4 bytes - 4x more! 😲

------

b) Columns `D_63` and `D_64` have strings instead of integers.

To speed up analytics and conserve memory, we will assign to each category a number.

We will use [StringIndexer](https://spark.apache.org/docs/latest/ml-features#stringindexer).
It's similar to [LabelEncoder from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

------

Now we will solve these problems. 💫

Notice, how schema changes after each step.

In [ ]:
def cast_categoricals_to_byte(df):    
    for c in categorical_columns - custom_label_columns:
        # these were written as floats, like "1.0"
        # however, they are in reality only small bytes
        df = df.withColumn(c, F.col(c).cast(pyspark.sql.types.ByteType()))
        
    return df

train_df = cast_categoricals_to_byte(train_df)
test_df = cast_categoricals_to_byte(test_df)

train_df

In [ ]:
# columns 'D_64' and 'D_63' have string labels instead of integers
# we replace them with integers

from pyspark.ml.feature import StringIndexerModel

# I've manually fitted StringIndexer.
# This indexer is there recreated with `from_arrays_of_labels` function
string_indexer = StringIndexerModel.from_arrays_of_labels(
    [['CL', 'CO', 'CR', 'XL', 'XM', 'XZ'], ['-1', 'O', 'R', 'U']],
    inputCols=['D_63', 'D_64'],
    outputCols=['D_63_index', 'D_64_index'],
    handleInvalid="keep",  # nulls and labels not in the set
)

def index_labels(df):
    df = string_indexer.transform(df)
    
    for c in custom_label_columns:        
        index_col = F.col(c + '_index')
        
        # give null on the output, when original input was also null
        index_col = F.when(F.col(c).isNull(), None).otherwise(index_col)
        
        # indexer returns floats, we want bytes
        index_col = index_col.cast(pyspark.sql.types.ByteType())
                
        df = df.withColumn(c + '_index', index_col)
        df = df.drop(c)
    
    return df

train_df = index_labels(train_df)
test_df = index_labels(test_df)

train_df

## 3. Export to Parquet

Now we will save the data to Parquet.

### Lazy evaluation

Up to this point *no computation was done*.
Spark was only building a query plan with *tranformations*, which will be run by an *action* - write to Parquet!

### Partitioning

We are using option `repartition`, which will cause the result of the computation to be written to many files.

Spark will make sure, that all records of a given customer will be in one partition.

Therefore, you can load in Pandas only one Parquet file - and be sure, that you see all
interactions of a given customer.

Notice, that each Parquet file will contain many customers.

There is [a blog post about repartitioning](https://medium.com/@vladimir.prus/spark-partitioning-the-fine-print-5ee02e7cb40b).

In [ ]:
%%time

def write_parquet(df, name, partitions):
    (
        df
        .repartition(partitions, "customer_ID")
        .sortWithinPartitions("customer_ID", "S_2")
        .write
        .parquet("amexparq/" + name)
    )
    
write_parquet(train_df, "train", 20)
write_parquet(test_df, "test", 40)

In [ ]:
!ls /kaggle/working/amexparq/train -lh

In [ ]:
!ls /kaggle/working/amexparq/test -lh

In [ ]:
!du -sh /kaggle/working/amexparq/*

## 4. Perform some trivial EDA 🔎

To wheat your appetite, we will do some basic EDA. You can **use it/fork it** as a basis of *your EDA*! 

It turns out, that Spark has an optional **pandas-like API**.

First, we read our *fresh parquet files* for an analysis:

In [ ]:
import pyspark.pandas as ps  # enable pandas-like API in PySpark

train_df = spark.read.parquet("/kaggle/working/amexparq/train")
test_df = spark.read.parquet("/kaggle/working/amexparq/test")
train_labels_df = spark.read.parquet("/kaggle/working/amexparq/train_labels").cache()

ps.set_option('compute.default_index_type', 'distributed')
# magic to speedup pandas-API in Spark
# see https://databricks.com/blog/2020/08/11/interoperability-between-koalas-and-apache-spark.html

### About Parquet/Spark performance

Spark has a few optimizing techniques.

If your query reads only column `D_63_index`, then other columns will
be skipped during Parquet file reading - reducing I/O!

In the next query we calculate, how often given `D_63_index` was used.

Notice, that in the physical plan we have a line:

```
    +- FileScan parquet [D_63_index#7492] Batched: ...,
```

The array of columns to read is `[D_63_index#7492]`.

Spark will read only `D_63_index` column!

In [ ]:
(
    train_df
    .select('D_63_index')
    .groupby('D_63_index')
    .count()
).explain()

In [ ]:
(
    train_df
    .select('D_63_index')
    .groupby('D_63_index')
    .count()
).show()

### Let's use pandas-API to do EDA!

In [ ]:
%%time

# train dataframe with pandas-API
train_pf = train_df.to_pandas_on_spark()

train_pf.agg(['min', 'max'])

**Histogram of `D_63_index`**

In [ ]:
%%time
train_pf['D_63_index'].plot.hist(bins=np.arange(7) - 0.5)

**Histogram of user interaction number**

In [ ]:
%%time
train_pf['customer_ID'].value_counts().plot.hist(bins=np.arange(100) - 0.5)

These plots are **interactive** - use mouse to zoom in the previous one!

## Conclusion

I hope you've enjoyed the notebook.

Resulting Parquet dataset will be published as **a Kaggle dataset**, so you can use it in your notebooks. 👍

## Questions? Improvement ideas?

Please comment! I will try to answer. 😁
